<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://bit.ly/2Ooh01X)


# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [4]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [23]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [24]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitutionusing the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [21]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)')

# Remember to commit the changes
conn.commit()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 5, and there are 0 supplied.

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [26]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses values (?,?,?,?,?)', recent_sales)
conn.commit()

Select all rows from houses

In [28]:
# A
for row in c.execute('SELECT * FROM houses ORDER BY price'):
        print (row)

(3, 1505, 3, 90, 269000)
(2, 1870, 3, 14, 289000)
(1, 2390, 4, 34, 319000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [42]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:

data = genfromtxt('C:\\Users\\praty\\Documents\\DSIA\\DSIA-SYD-FT-Projects-201907\\Sarita\\Labs\\DATA2\\housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [43]:
data

[[2104, 3, 70, 399900],
 [1600, 3, 28, 329900],
 [2400, 3, 44, 369000],
 [1416, 2, 49, 232000],
 [3000, 4, 75, 539900],
 [1985, 4, 61, 299900],
 [1534, 3, 12, 314900],
 [1427, 3, 57, 198999],
 [1380, 3, 14, 212000],
 [1494, 3, 15, 242500],
 [1940, 4, 7, 239999],
 [2000, 3, 27, 347000],
 [1890, 3, 45, 329999],
 [4478, 5, 49, 699900],
 [1268, 3, 58, 259900],
 [2300, 4, 77, 449900],
 [1320, 2, 62, 299900],
 [1236, 3, 78, 199900],
 [2609, 4, 5, 499998],
 [3031, 4, 21, 599000],
 [1767, 3, 44, 252900],
 [1888, 2, 79, 255000],
 [1604, 3, 13, 242900],
 [1962, 4, 53, 259900],
 [3890, 3, 36, 573900],
 [1100, 3, 60, 249900],
 [1458, 3, 29, 464500],
 [2526, 3, 13, 469000],
 [2200, 3, 28, 475000],
 [2637, 3, 25, 299900],
 [1839, 2, 40, 349900],
 [1000, 1, 5, 169900],
 [2040, 4, 75, 314900],
 [3137, 3, 67, 579900],
 [1811, 4, 24, 285900],
 [1437, 3, 50, 249900],
 [1239, 3, 22, 229900],
 [2132, 4, 28, 345000],
 [4215, 4, 66, 549000],
 [2162, 4, 43, 287000],
 [1664, 2, 40, 368500],
 [2238, 3, 37, 3299

In [44]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None) #insering none in the index 0, 1st column

In [45]:
data

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [46]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [49]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [50]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

In [51]:
data # you dont see none in the sqlite as it is incrementing none+1 every time

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [52]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

In [53]:
data

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

QUIZ: What would `DELETE FROM houses` do? it has deleted rows 52 and 53 from field 1

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [62]:
# ANSWER
c.execute('SELECT * FROM houses WHERE bdrms=4').fetchall() #it needs fetchall() statement

[(1, 2390, 4, 34, 319000),
 (8, 3000, 4, 75, 539900),
 (9, 1985, 4, 61, 299900),
 (14, 1940, 4, 7, 239999),
 (19, 2300, 4, 77, 449900),
 (22, 2609, 4, 5, 499998),
 (23, 3031, 4, 21, 599000),
 (27, 1962, 4, 53, 259900),
 (36, 2040, 4, 75, 314900),
 (38, 1811, 4, 24, 285900),
 (41, 2132, 4, 28, 345000),
 (42, 4215, 4, 66, 549000),
 (43, 2162, 4, 43, 287000),
 (46, 2567, 4, 57, 314000),
 (49, 1852, 4, 64, 299900),
 (56, 3000, 4, 75, 539900),
 (57, 1985, 4, 61, 299900),
 (62, 1940, 4, 7, 239999),
 (67, 2300, 4, 77, 449900),
 (70, 2609, 4, 5, 499998),
 (71, 3031, 4, 21, 599000),
 (75, 1962, 4, 53, 259900),
 (84, 2040, 4, 75, 314900),
 (86, 1811, 4, 24, 285900),
 (89, 2132, 4, 28, 345000),
 (90, 4215, 4, 66, 549000),
 (91, 2162, 4, 43, 287000),
 (94, 2567, 4, 57, 314000),
 (97, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [63]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1807.8653846153845, 320161.46153846156),
 (4, 2400.0689655172414, 375434.275862069),
 (5, 4478.0, 699900.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [64]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [65]:
data = pd.read_csv('C:\\Users\\praty\\Documents\\DSIA\\DSIA-SYD-FT-Projects-201907\\Sarita\\Labs\\DATA2\\housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [67]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [69]:
#ANSWER
c.execute("SELECT sqft, AVG(price) AS avg_price FROM houses GROUP BY sqft")
results.fetchall()

[(852, 179900.0),
 (1000, 169900.0),
 (1100, 249900.0),
 (1200, 274500.0),
 (1203, 239500.0),
 (1236, 199900.0),
 (1239, 229900.0),
 (1268, 259900.0),
 (1320, 299900.0),
 (1380, 212000.0),
 (1416, 232000.0),
 (1427, 198999.0),
 (1437, 249900.0),
 (1458, 464500.0),
 (1494, 242500.0),
 (1505, 269000.0),
 (1534, 314900.0),
 (1600, 329900.0),
 (1604, 242900.0),
 (1664, 368500.0),
 (1767, 252900.0),
 (1811, 285900.0),
 (1839, 349900.0),
 (1852, 299900.0),
 (1870, 289000.0),
 (1888, 255000.0),
 (1890, 329999.0),
 (1940, 239999.0),
 (1962, 259900.0),
 (1985, 299900.0),
 (2000, 347000.0),
 (2040, 314900.0),
 (2104, 399900.0),
 (2132, 345000.0),
 (2162, 287000.0),
 (2200, 475000.0),
 (2238, 329900.0),
 (2300, 449900.0),
 (2390, 319000.0),
 (2400, 369000.0),
 (2526, 469000.0),
 (2567, 314000.0),
 (2609, 499998.0),
 (2637, 299900.0),
 (3000, 539900.0),
 (3031, 599000.0),
 (3137, 579900.0),
 (3890, 573900.0),
 (4215, 549000.0),
 (4478, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [22]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [71]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:\\Users\\praty\\Documents\\DSIA\\DSIA-SYD-FT-Projects-201907\\Sarita\\Labs\\DATA2\\P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:\\Users\\praty\\Documents\\DSIA\\DSIA-SYD-FT-Projects-201907\\Sarita\\Labs\\DATA2\\P12-OrderBreakdown.csv', encoding = 'utf-8')

In [72]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [73]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [74]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [75]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [76]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [77]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [ ]:
##push the dataframe to sql 
#my_data.to_sql("my_data", conn, if_exists="replace")

##create the table

#conn.execute(
    #
    #create table my_table as 
    #select * from my_data
    """)

In [143]:
db_connection=sqlite3.connect("eshop.db.sqlite")

In [85]:
orders.to_sql('ORDR', con=sqlite3.connect("eshop.db.sqlite"), index=False) #be careful create both tables to be able to use
                                                                           for extraction


In [94]:
orders_break_down.to_sql('brkdwn', con=sqlite3.connect("eshop.db.sqlite"), index=False)

#### 5. Select Number of Orders for Each Customer

In [95]:
# Select Number of Orders for Each Customer
sqlite3.connect("eshop.db.sqlite").execute("SELECT * FROM ORDR").fetchall()



[('IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden'),
 ('ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom'),
 ('ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France'),
 ('IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom'),
 ('ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France'),
 ('ES-2011-5433855', '2011-01-07', 'Dave Poirier', 'France'),
 ('IT-2011-4546695', '2011-01-08', 'Darren Powers', 'France'),
 ('ES-2011-1138719', '2011-01-11', 'Eric Murdock', 'Italy'),
 ('ES-2011-1466305', '2011-01-11', 'Mick Brown', 'Austria'),
 ('ES-2011-4359424', '2011-01-11', 'Dorothy Dickinson', 'Spain'),
 ('ES-2011-5158390', '2011-01-11', 'Roy Collins', 'United Kingdom'),
 ('ES-2011-1460199', '2011-01-12', 'Roy Phan', 'Germany'),
 ('ES-2011-4976144', '2011-01-12', 'Anthony Garverick', 'United Kingdom'),
 ('ES-2011-5040255', '2011-01-12', 'Peter B?hler', 'United Kingdom'),
 ('IT-2011-5340302', '2011-01-13', 'Greg Matthias', 'Netherlands'),
 ('IT-2

In [125]:
sql_string = '''SELECT 
        ORDR.customer_name, 
        COUNT(DISTINCT ORDR.order_id) AS Count 
    FROM ORDR 
    JOIN brkdwn 
    ON brkdwn.order_id = ORDR.order_id
    GROUP BY ORDR.customer_name 
    ORDER BY Count DESC'''

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

,customer_name,Count
0,John Grady,13
1,Aaron Smayling,12
2,Joel Jenkins,12
3,Aaron Bergman,11
4,Anthony Rawles,11


In [126]:
#Number of Orders for Each Customer
sqlite3.connect("eshop.db.sqlite").execute("SELECT order_id, customer_name FROM ORDR").fetchall()#so it is from first table ORDR


[('IT-2011-3647632', 'Eugene Moren'),
 ('ES-2011-4869686', 'Dorothy Dickinson'),
 ('ES-2011-4939443', 'Arthur Prichep'),
 ('IT-2011-2942451', 'Grant Thornton'),
 ('ES-2011-3848439', 'Michael Granlund'),
 ('ES-2011-5433855', 'Dave Poirier'),
 ('IT-2011-4546695', 'Darren Powers'),
 ('ES-2011-1138719', 'Eric Murdock'),
 ('ES-2011-1466305', 'Mick Brown'),
 ('ES-2011-4359424', 'Dorothy Dickinson'),
 ('ES-2011-5158390', 'Roy Collins'),
 ('ES-2011-1460199', 'Roy Phan'),
 ('ES-2011-4976144', 'Anthony Garverick'),
 ('ES-2011-5040255', 'Peter B?hler'),
 ('IT-2011-5340302', 'Greg Matthias'),
 ('IT-2011-5951216', 'Stuart Calhoun'),
 ('ES-2011-1043483', 'Sharelle Roach'),
 ('ES-2011-3661686', 'Don Jones'),
 ('IT-2011-1373680', 'Shirley Daniels'),
 ('ES-2011-4174897', 'Michael Dominguez'),
 ('ES-2011-4825230', 'Richard Bierner'),
 ('ES-2011-1690587', 'Ann Blume'),
 ('ES-2011-3666932', 'Matt Abelman'),
 ('ES-2011-1122594', 'Darren Koutras'),
 ('ES-2011-3229667', 'Ryan Crowe'),
 ('ES-2011-3906248', 'M

In [129]:
# in order to combine the two tables to be able to extract the data from both we need to join them using join command,
#left join is better because we want orderid for customers ordr has both columns and brkdwn has only orderid, left join
#will extract everything from ORDR from left and orderid from brkdwn on the right

#FROM table1
#LEFT JOIN table2
#ON table1.column_name = table2.column_name;

sqlite3.connect("eshop.db.sqlite").execute('''SELECT order_id, 
                                           FROM ORDR LEFT JOIN brkdwn 
                                           ON ORDR.order_id=brkdwn.order_id''')


OperationalError: near "FROM": syntax error

#### 6. Select Number of Customers for Each Country

In [130]:
# Select Number of Customers for Each Country
sql_string = '''
    SELECT 
        country, 
        COUNT(DISTINCT customer_name) AS Count 
    FROM ORDR 
    GROUP BY country 
    ORDER BY Count DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,country,Count
0,France,568
1,Germany,505
2,United Kingdom,473
3,Italy,351
4,Spain,306


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [132]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
sql_string = '''
    SELECT 
        discount
        , sales
        , quantity 
    FROM brkdwn
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,discount,sales,quantity
0,0.5,45.0,3
1,0.0,854.0,7
2,0.0,140.0,3
3,0.5,27.0,2
4,0.5,17.0,2


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [133]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
sql_string = '''
    SELECT 
        discount
        , sales
        , quantity
        , (sales * quantity - discount) AS total_price 
    FROM brkdwn
    ORDER BY total_price DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,discount,sales,quantity,total_price
0,0.1,5729.0,14,80205.9
1,0.0,6517.0,12,78204.0
2,0.1,4748.0,14,66471.9
3,0.1,5726.0,10,57259.9
4,0.0,4141.0,13,53833.0


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [135]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
sql_string = '''
    SELECT 
        *
        , (sales * quantity - discount) AS total_price
    FROM brkdwn
    WHERE sales * quantity - discount > 100
    ORDER BY total_price ASC

'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-2668777,"Binney & Smith Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
1,IT-2011-1165265,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
2,ES-2012-1763830,"Elite Scissors, High Speed",0.5,34.0,3,Office Supplies,101.5
3,IT-2012-4099397,"Sanford Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
4,ES-2013-4250046,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5


#### 8. Select All Customer And The Product They Have Bought

In [136]:
# Select All Customer And The Product They Have Bought
sql_string = '''
    SELECT 
        ORDR.order_id
        , ORDR.customer_name
        , brkdwn.product_name
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
''' 
customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [137]:
# Select Number of 'Furniture' Orders For Each Country
sql_string = '''
    SELECT 
        ORDR.country
        , COUNT(brkdwn.category) AS Count           
    FROM ORDR
    JOIN brkdwn
    ON ORDR.order_id = brkdwn.order_id
    WHERE brkdwn.category = 'Furniture'
    GROUP BY ORDR.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,Count
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [138]:
# Select Number of 'Furniture' Orders For The Country Denmark
sql_string = '''
    SELECT 
        *       
    FROM ORDR
    JOIN brkdwn
    ON ORDR.order_id = brkdwn.order_id
    WHERE brkdwn.category = 'Furniture'
        AND ORDR.country = 'Denmark'
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()


,order_id,order_date,customer_name,country,order_id,product_name,discount,sales,quantity,category
0,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture
1,IT-2012-4084224,2012-02-15,Mick Brown,Denmark,IT-2012-4084224,"Rubbermaid Door Stop, Ergonomic",0.6,17.0,1,Furniture
2,IT-2013-4649004,2013-05-31,Bart Folk,Denmark,IT-2013-4649004,"Novimex Executive Leather Armchair, Red",0.5,920.0,4,Furniture
3,ES-2013-3422294,2013-09-07,Dave Brooks,Denmark,ES-2013-3422294,"Advantus Door Stop, Duo Pack",0.6,36.0,2,Furniture
4,ES-2014-3653593,2014-02-10,Ivan Liston,Denmark,ES-2014-3653593,"Deflect-O Clock, Black",0.6,161.0,8,Furniture


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [140]:
# Select Total Sales With Discount and Without Discount for Each Country
sql_string = '''
    SELECT
        ORDR.country
        , SUM(CASE WHEN discount == 0 THEN sales ELSE 0 END) AS discount_sales
        , SUM(CASE WHEN discount > 0 THEN sales ELSE 0 END) AS non_discount_sales       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    GROUP BY ORDR.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,discount_sales,non_discount_sales
0,Austria,79382.0,0.0
1,Belgium,42320.0,0.0
2,Denmark,0.0,7763.0
3,Finland,20702.0,0.0
4,France,129791.0,479892.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [145]:
# Select Total Quantity, Total Sales for Each Country
sql_string = '''
    SELECT
        ORDR.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    GROUP BY ORDR.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329


#### 11.B Select Top 3 Country Based on Sales

In [158]:
# Select Top 3 Country Based on Sales
sql_string = '''
    SELECT
        ORDR.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    GROUP BY ORDR.country
     ORDER BY total_sales DESC LIMIT 3'''
order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


In [159]:
order_list.sort_values(by='total_sales', ascending=False).head(3) #another way of executing the above

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


#### 11.C Select Bottom 3 Country Based On Quantities

In [148]:
# Select Bottom 3 Country Based On Quantities
sql_string = '''
    SELECT
        ORDR.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    GROUP BY ORDR.country
     ORDER BY total_quantity DESC LIMIT 3'''
order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


In [149]:
order_list.sort_values(by='total_sales', ascending=False).head(3) #another way oif executing the above

,country,total_sales,total_quantity
0,France,609683.0,7329
1,Germany,488681.0,6179
2,United Kingdom,420497.0,4917


#### 12. Select Average Sales By Categroy For The Country 'France'

In [167]:
sql_string = '''
    SELECT
        ORDR.country
        , brkdwn.category
        , AVG(sales) AS avg_sales       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    WHERE ORDR.country = 'France'
    GROUP BY ORDR.country, brkdwn.category
    ORDER BY avg_sales DESC
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head(100)

,country,category,avg_sales
0,France,Technology,595.145078
1,France,Furniture,582.314381
2,France,Office Supplies,167.217709


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [168]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
sql_string = '''
    SELECT
        ORDR.country
        , brkdwn.category
        , AVG(sales) AS total_sales       
    FROM ORDR
    JOIN brkdwn ON ORDR.order_id = brkdwn.order_id
    GROUP BY ORDR.country, brkdwn.category
    ORDER BY total_sales DESC LIMIT 1
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

,country,category,total_sales
0,Switzerland,Technology,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax





> 
> 
>
>
>
>
>

> 


> 



> 












**© 2019 Data Science Institute of Australia**